# Data Mining
### Project for Data Mining, Spring 2019


Contributors
------------

* [Sofia Kostakonti](https://github.com/SofiaKstk/ "Sofia Kostakonti") : 1115201500080
* [Ioannis Pelekoudas](https://github.com/pelekoudasq/ "Ioannis Pelekoudas") : 1115201500128

#### First Part - Clean Data

Αφαιρούμε τα σύμβολα, όπως hashtags, emoticons, τα links και τα stopwords από το training set.

#### Second Part - Tokenization

Διαχωρισμός των λέξεων.

In [1]:
import pandas as pd
import numpy as np
from emoji import UNICODE_EMOJI
import re
import nltk
import string
import sklearn
import pickle

In [2]:
DataLocation = r'./twitter_data/test2017.tsv'

df = pd.read_csv(DataLocation, sep='\t', header=None, usecols=[3])
sd = df
sd.columns = ["tweets"]

from nltk.tokenize import TweetTokenizer
tkn = TweetTokenizer()
tokens = tkn.tokenize(' '.join(sd['tweets']))

from nltk.corpus import stopwords
stop = stopwords.words("english")

tokens_clean1 = [word.lower() for word in tokens if word not in string.punctuation and word != '...' and word!='..']
tokens_stop = [word for word in tokens_clean1 if word not in (stop)]
tokens_clean = [re.sub(r'^@.*', '@user', word) for word in tokens_stop]
tokens_clean = [re.sub(r'^#.*', '#hashtag', word) for word in tokens_clean]
tokens_clean = [re.sub(r'[0-9]+', '$number', word) for word in tokens_clean]
tokens_clean = [re.sub(r'(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', '%url', word) for word in tokens_clean]
#tokens_clean = [for word in tokens_clean: '<emoji>' if word in UNICODE_EMOJI else word]


In [3]:
with open ('clean_test', 'wb') as file:
    pickle.dump(tokens_clean, file)

## Στατιστικά και δεδομένα για τις λέξεις όλων των tweets συνολικά.

##### Wordcloud για όλες τις λέξεις.

In [ ]:
#python -m pip install --user wordcloud

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

with open('clean_test', 'rb') as file:
    tokens_clean = pickle.load(file)

texts = {}
texts = ' '.join(tokens_clean)
my_stopwords = {'hashtag', 'user', 'number', 'url', 'in', 'is', 'that', 'it', 'and', 'for', 'he', 'i\'m', 'to', 'you', 'this', 'the'}
stop_words = STOPWORDS.union(my_stopwords)

wordcloud = WordCloud(max_font_size=200, min_font_size=10, max_words=500, stopwords=stop_words, background_color="white", relative_scaling=.4, height=800, width=1600).generate(texts)
plt.figure(figsize=(60,40))
plt.title('All Words')
plt.imshow(wordcloud)
plt.axis('off')


(-0.5, 1599.5, 799.5, -0.5)

Χωρίζουμε τις λέξεις σε θετικές και αρνητικές σύμφωνα με το λεξικό που έχει δωθεί προκειμένου να πάρουμε κάποια στατιστικά στοιχεία.

In [ ]:
Lexica = r'./lexica/affin/affin.txt'

lex = pd.read_csv(Lexica, sep="\t", header=None)
lex.columns = ["word", "weight"]

positive=[]
negative=[]
non=[]

hasht = users = num = url = 0

for w in tokens_clean:
    if '#hashtag' in w:
        hasht += 1
    elif '@user' in w:
        users += 1
    elif '$number' in w:
        num += 1
    elif '%url' in w:
        url += 1
    else:
        flag = 0
        for i in range(len(lex)):
            if w == lex['word'][i]:
                flag = 1
                if lex['weight'][i] < 0:
                    negative.append(w)
                else:
                    positive.append(w)
        if flag == 0:
            non.append(w)


#### Στατιστικά για hashtags, usertags, urls και αριθμούς.

In [ ]:
hashper = round(hasht / len(tokens_clean) *100)
userper = round(users / len(tokens_clean) *100)
numper = round(num / len(tokens_clean) *100)
urlper = round(url / len(tokens_clean) *100)
print ('Hashtags    Usertags     Numbers     Urls')
print (str(hashper)+ '%         '+str(userper)+ '%           '+str(numper)+ '%          '+str(urlper)+'%')

 ##### Wordcloud για τις θετικές λέξεις.

In [ ]:
pos = ' '.join(positive)
cloudPos = WordCloud(max_font_size=200, min_font_size=10, max_words=500, stopwords=stop_words, background_color="white", relative_scaling=.4, height=800, width=1600).generate(pos)
plt.figure(figsize=(30, 20))
plt.title('Positive Words')
plt.imshow(cloudPos)
plt.axis('off')

##### Wordcloud για τις αρνητικές λέξεις.

In [ ]:
neg = ' '.join(negative)
cloudNeg = WordCloud(max_font_size=200, min_font_size=10, max_words=500, stopwords=stop_words, background_color="white", relative_scaling=.4, height=800, width=1600).generate(neg)
plt.figure(figsize=(30, 20))
plt.title('Negative Words')
plt.imshow(cloudNeg)
plt.axis('off')

##### Wordcloud για όλες τις υπόλοιπες λέξεις που δεν υπάρχουν στο λεξικό.

In [ ]:
nan = ' '.join(non)
cloudNan = WordCloud(max_font_size=200, min_font_size=10, max_words=500, stopwords=stop_words, background_color="white", relative_scaling=.4, height=800, width=1600).generate(nan)
plt.figure(figsize=(30, 20))
plt.title('Other Words')
plt.imshow(cloudNan)
plt.axis('off')

## Στατιστικά και δεδομένα για θετικά, αρνητικά και ουδέτερα tweets ξεχωριστά.

Παίρνουμε τα αποτελέσματα των tweets από το αρχείο gold και βγάζουμε κάποια συμπεράσματα.

In [ ]:
Gold = r'./twitter_data/SemEval2017_task4_subtaskA_test_english_gold.txt'

res = pd.read_csv(Gold, sep="\t", header=None)
#res.columns = ["word", "weight"]
print (len(df))
print (len(res))

## Categorizing

In [ ]:
trainSet = pd.read_csv(r'./twitter_data/train2017.tsv', sep='\t', header=None, usecols=[2,3])
testSet = pd.read_csv(r'./twitter_data/test2017.tsv', sep='\t', header=None, usecols=[3])
trainSet.columns = ["review", "tweets"]
testSet.columns = ['tweets']
small = trainSet

from nltk.corpus import stopwords
stop = stopwords.words("english")

train_tweets = []
for t in small['tweets']:
    tweet = t.split()
    tokens_clean1 = [word.lower() for word in tweet if word not in string.punctuation and word != '...' and word!='..']
    tokens_stop = [word for word in tokens_clean1 if word not in (stop)]
    tokens_clean = [re.sub(r'^@.*', '@user', word) for word in tokens_stop]
    tokens_clean = [re.sub(r'^#.*', '#hashtag', word) for word in tokens_clean]
    tokens_clean = [re.sub(r'[0-9]+', '$number', word) for word in tokens_clean]
    tokens_clean_train = [re.sub(r'(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', '%url', word) for word in tokens_clean]
    t = ' '.join(tokens_clean_train)
    train_tweets.append(t)

test_tweets = []
for t in testSet['tweets']:
    tweet = t.split()
    tokens_clean1 = [word.lower() for word in tweet if word not in string.punctuation and word != '...' and word!='..']
    tokens_stop = [word for word in tokens_clean1 if word not in (stop)]
    tokens_clean = [re.sub(r'^@.*', '@user', word) for word in tokens_stop]
    tokens_clean = [re.sub(r'^#.*', '#hashtag', word) for word in tokens_clean]
    tokens_clean = [re.sub(r'[0-9]+', '$number', word) for word in tokens_clean]
    tokens_clean_train = [re.sub(r'(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', '%url', word) for word in tokens_clean]
    t = ' '.join(tokens_clean_train)
    test_tweets.append(t)

with open ('review_train', 'wb') as file:
    pickle.dump(small['review'], file)
with open ('clean_train', 'wb') as file:
    pickle.dump(train_tweets, file)
with open ('clean_test', 'wb') as file:
    pickle.dump(test_tweets, file)

In [ ]:
with open('clean_train', 'rb') as file:
    train_tweets = pickle.load(file)
with open('clean_test', 'rb') as file:
    test_tweets = pickle.load(file)

### Bag of Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer(max_df=1.0, min_df=1, max_features=1000, stop_words=stop)
bow_xtrain = bow_vectorizer.fit_transform(train_tweets)
bow_xtest = bow_vectorizer.fit_transform(test_tweets)


### TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, max_features=1000, stop_words=stop)
tfidf_xtrain = tfidf_vectorizer.fit_transform(train_tweets)
tfidf_xtest = tfidf_vectorizer.fit_transform(test_tweets)

### Word Embedding

In [ ]:
from nltk.tokenize import TweetTokenizer
tkn = TweetTokenizer()

tokenized_train = tkn.tokenize(' '.join(train_tweets))
tokenized_test = tkn.tokenize(' '.join(test_tweets))

tokens = tokenized_train + tokenized_test

import gensim

model_w2v = gensim.models.Word2Vec(
            tokens,
            size=200, # desired no. of features/independent variables
            window=5, # context window size
            min_count=2,
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 2, # no.of cores
            seed = 34) 

model_w2v.train(tokens, total_examples=(len(train_tweets) + len(test_tweets)), epochs=20)


In [ ]:
with open('review_train', 'rb') as file:
    ytrain = pickle.load(file)
    
ytest = pd.read_csv(r'./twitter_data/SemEval2017_task4_subtaskA_test_english_gold.txt', sep='\t', header=None, usecols=[1])


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(bow_xtrain.toarray(), ytrain)

In [ ]:
predict_bow_knn = knn.predict(bow_xtest.toarray())

from sklearn.metrics import accuracy_score 

print (accuracy_score(ytest, predict_bow_knn))
# accuracy with 3 neighbors = 0.43121133181374144

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(tfidf_xtrain.toarray(), ytrain)
predict_tfidf_knn = knn.predict(tfidf_xtest.toarray())

from sklearn.metrics import accuracy_score 

print (accuracy_score(ytest, predict_tfidf_knn))
# accuracy with 3 neighbors = 0.43121133181374144

In [ ]:
from sklearn import svm

svc = svm.SVC(kernel='linear', C=1, probability=True)
svc.fit(bow_xtrain.toarray(), ytrain)
predict_bow_svm = svc.predict(bow_xtest.toarray())

from sklearn.metrics import accuracy_score 

print (accuracy_score(ytest, predict_bow_svm))
# accuracy = 03990556821882123

In [ ]:
from sklearn import svm

svc = svm.SVC(kernel='linear', C=1, probability=True)
svc.fit(tfidf_xtrain.toarray(), ytrain)
predict_tfidf_svm = svc.predict(tfidf_xtest.toarray())

from sklearn.metrics import accuracy_score 

print (accuracy_score(ytest, predict_tfidf_svm))
# accuracy = 03990556821882123